In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi as YTtrscrpt
#YTtscptAPI.get_transcript("9P6rdqiybaw")

videoID = "9P6rdqiybaw"

In [ ]:
VidLang = YTtrscrpt.get_transcript(videoID)
print(VidLang)

In [ ]:
VidLang = YTtrscrpt.list_transcripts(videoID)
#for lang in VidLang:
#    print(lang)


for lang in VidLang:
    fullLanguage = lang.language
    print(fullLanguage)
    code = lang.language_code
    print(code)

    test = lang.fetch
    print(test)

    




In [ ]:
#Readme
# Responses - JSON
'''
 "data": Mixed container, holding the data

'''

'''data
    {
        "videoID": "9P6rdqiybaw",
        "videoLang": "us-en",
        "transcipt": "JSON-FILE"
    }
'''


In [ ]:
#server
from flask import Flask, json

testitem = [{"VideoID": "9P6rdqiybaw", "videoLang": "us-en", "transcript": "jsonFiletoAdd"}]
serv = Flask(__name__)

@serv.route('/getVideoTranscript', methods=['GET'])
def get_VideoTranscript():
    return json.dumps(YTtrscrpt.get_transcript("9P6rdqiybaw"))

if __name__ == '__main__':
    serv.run()

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler
from youtube_transcript_api import YouTubeTranscriptApi as YTtrscrpt
import urllib.parse as urlparse

import json



#http://www.compciv.org/guides/python/how-tos/creating-proper-url-query-strings/

params = {
    'vidID': '',
    'vidLang': ''
}

class getVideoScript(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        print(self.headers)
        #query fetch
        print("Querry:")
        url_path = urlparse.urlparse(self.path)
        #print(url_path.query)
        try:
            query_components = dict(qc.split("=") for qc in url_path.query.split("&"))
            print(query_components)
        except: 
            print("No Querries found! Or No idea?")




        #self.send_header('content-type', 'text/html')
        #self.send_header('content-type', 'json/xml')
        self.send_header('content-type', 'application/json')
        self.end_headers()
        #self.wfile.write("Hello World!".encode())
        
        self.wfile.write(json.dumps(YTtrscrpt.get_transcript("9P6rdqiybaw")).encode())
        #self.wfile.write((YTtrscrpt.list_transcripts("9P6rdqiybaw")).encode())

        
def main():
    PORT = 9889
    server = HTTPServer(('', PORT), getVideoScript)
    print('Server running on port %s.' % PORT)
    server.serve_forever()

if __name__ == "__main__":
    main()

In [ ]:
#test URL: http://localhost:9988/?vidID=9P6rdqiybaw&vidLangCode=en

from http.server import HTTPServer, BaseHTTPRequestHandler
from youtube_transcript_api import YouTubeTranscriptApi as YTTranscriptAPI
import urllib.parse as urlparse
import json

params = {
    "vidID": "",
    "vidLangCode" : ""
}

def returnVideoLangages(vidID):
    try:
        languages = YTTranscriptAPI.list_transcripts(vidID)
    except Exception as e:
        print(e)
        print("VidID: " + vidID)

    ret = []
    for lang in languages:
        ret.append({
            'code': lang.language_code,
            'name': lang.language,
            'isGenerated': lang.is_generated,
            'isTranslatable': lang.is_translatable
            })
    return ret

def returnVideoScript(vidID, LangCode):
    try:
        ret = YTTranscriptAPI.get_transcript(vidID, languages=[LangCode])
    except Exception as e:
        print(e)
        print("VidID: " + vidID)
        print("Language Code: " + LangCode)
        return -1
    return ret



#server
class YTScriptGrabbingService(BaseHTTPRequestHandler):
    def do_GET(self):
        url_path = urlparse.urlparse(self.path)
        try:
            request_query = dict(qc.split("=") for qc in url_path.query.split("&"))
        except:
            #send error
            self.send_response(400)
            return
        



        #check what response should be given
        if {'vidID', 'vidLangCode'} <= request_query.keys():
            #both keys are present
            print("Both Keys are present...")
            jsonFile = returnVideoScript(request_query.get('vidID'), request_query.get('vidLangCode'))
            if(jsonFile == -1): 
                #error has encured
                self.send_response(400)
                return
            self.send_response(200)
            self.send_header('content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(jsonFile).encode())
            return

        if {'vidID'} <=request_query.keys():
            #only the Video ID is present
            print("Key vidID is present...")
            jsonFile = returnVideoLangages(request_query.get('vidID'))
            if(jsonFile == -1):
                #error has encured
                self.send_response(400)
                return
            self.send_response(200)
            self.send_header('content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(jsonFile).encode())
            return
        
        #either only vidLangCode is present or a wrong query is present
        self.send_response(400)
        return

def main():
    PORT = 9988
    server = HTTPServer(('', PORT), YTScriptGrabbingService)
    print('Server running on port %s.' % PORT)
    server.serve_forever()

if __name__ == "__main__":
    main()